In [1]:
import matplotlib.pyplot as plt
import gymnasium as gym
from evaluate import evaluate_HIV, evaluate_HIV_population
from train import ProjectAgent  # Replace DummyAgent with your agent implementation
from statistics import mean
from functools import partial
from gymnasium.wrappers import TimeLimit
from env_hiv import HIVPatient
from interface import Agent
import random
import numpy as np
import torch
import torch.nn as nn
from copy import deepcopy
import math
import random
import matplotlib
import time
from collections import namedtuple
from itertools import count
from PIL import Image
import torch.optim as optim
import torch.nn.functional as F
import torchvision.transforms as T
from torch.utils.tensorboard import SummaryWriter
from torch.autograd import Variable
from torch.distributions import Categorical
import ffmpeg
import seaborn as sns
import os
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
from dqn_agent import ReplayBuffer, greedy_action, evaluate_agent, dqn_agent
from dqn_agent_general import dqn_agent_general, evaluate_agent_general
from dqn_agent_2 import dqn_agent_2, evaluate_agent_2

In [3]:
def seed_everything(seed: int = 42):
    random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    torch.cuda.manual_seed_all(seed)

In [24]:
##SAME TEST AS THE ONE OF THE EXAM##
def test_as_exam(Agent_class = dqn_agent_2, load_path="best_model_standard_scaling_general.pth", reward_path="best_deterministic_reward_2.pth"):
    env = TimeLimit(HIVPatient(domain_randomization=True), max_episode_steps=200)
    seed_everything(42)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    # Declare network
    state_dim = env.observation_space.shape[0]
    n_action = env.action_space.n 
    nb_neurons=128
    config = {'nb_actions': n_action, 'learning_rate': 0.01,
            'gamma': 0.95, 'buffer_size': 100_000,
            'epsilon_min': 0.01, 'epsilon_max': 1,
            'epsilon_decay_period': 5_000, 'epsilon_delay_decay': 600,
            'batch_size': 32, 'model_path': 'best_model_standard_scaling_general.pth',
            'scaling': 'standard', 'criterion': torch.nn.SmoothL1Loss()}
    DQN = torch.nn.Sequential(nn.Linear(state_dim, nb_neurons),
                            nn.ReLU(),
                            nn.Linear(nb_neurons, nb_neurons),
                            nn.ReLU(), 
                            nn.Linear(nb_neurons, n_action)).to(device)

    DQN.load_state_dict(torch.load(load_path, weights_only=True, map_location=device))
    print("Reward theoretical", torch.load(reward_path, weights_only=False, map_location=device))
    agent = Agent_class(config, DQN, load=True)

    score_agent_dr = evaluate_agent(agent, env, nb_episode = 20, scaling='standard')
    env = TimeLimit(HIVPatient(), max_episode_steps=200)
    score_agent = evaluate_agent(agent, env, nb_episode = 1, scaling='standard')
    print(f"Reward pb1: {score_agent/1_000_000:.2f}M")
    print(f"Reward pb2: {score_agent_dr/1_000_000:.2f}M")


In [28]:
test_as_exam(load_path="..\\best_model_standard_scaling_general.pth", reward_path="..\\best_deterministic_reward_2.pth")

Reward theoretical 10204.509430211989
Reward pb1: 8932.88M
Reward pb2: 11576.35M


In [30]:
test_as_exam(load_path="best_model_standard_scaling_general.pth", reward_path="best_deterministic_reward_2.pth")

Reward theoretical 15936.208078343408
Reward pb1: 2297.55M
Reward pb2: 9051.92M


In [23]:
test_as_exam(Agent_class = dqn_agent, load_path="best_model_standard_scaling.pth", reward_path="best_deterministic_reward.pth")

22410.298104915328
Score with domain randomization: 2123.16M
Score without domain randomization: 20235.54M
